In [1]:
!pip install scikit-learn
!pip install numpy==1.16.1

     |████████████████████████████████| 17.3MB 200kB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.2
    Uninstalling numpy-1.18.2:
      Successfully uninstalled numpy-1.18.2


In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/')
!ls
os.chdir('/content/drive/My Drive/Computer_vision_project/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
'My Drive'
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'My Drive'


In [3]:
## Libraries
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st
import sys
import warnings
warnings.filterwarnings('ignore')
np.random.seed(12227)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
def DataPreparation(data_input_file):
  print('Proposed Model {}'.format(data_input_file))
  data = np.load(data_input_file)
  X = data['X']
  X = X[:, 0, :, :]
  Y = data['y']
  folds = data['folds']
  classes_number = Y.shape[1]
  print(classes_number)
  Y = np.argmax(Y, axis=1)
  return X,Y,folds,classes_number

In [0]:
def A(sample):
    feat = []
    for col in range(0,sample.shape[1]):
        average = np.average(sample[:,col])
        feat.append(average)

    return feat

def SD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        std = np.std(sample[:, col])
        feat.append(std)

    return feat

def AAD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        add = np.mean(np.absolute(data - np.mean(data)))
        feat.append(add)

    return feat

def ARA(sample):
    #Average Resultant Acceleration[1]:
    # Average of the square roots of the sum of the values of each axis squared √(xi^2 + yi^2+ zi^2) over the ED
    feat = []
    sum_square = 0
    sample = np.power(sample, 2)
    for col in range(0, sample.shape[1]):
        sum_square = sum_square + sample[:, col]

    sample = np.sqrt(sum_square)
    average = np.average(sample)
    feat.append(average)
    return feat

def TBP(sample):
    from scipy import signal
    feat = []
    sum_of_time = 0
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        peaks = signal.find_peaks_cwt(data, np.arange(1,4))

        feat.append(peaks)

    return feat

In [0]:
## Feature Extraction

def feature_extraction(X):
    # Extracts the features, as mentioned by Catal et al. 2015
    # Average - A,
    # Standard Deviation - SD,
    # Average Absolute Difference - AAD,
    # Average Resultant Acceleration - ARA(1),
    # Time Between Peaks - TBP
    X_tmp = []
    for sample in X:
        features = A(sample)
        features = np.hstack((features, A(sample)))
        features = np.hstack((features, SD(sample)))
        features = np.hstack((features, AAD(sample)))
        features = np.hstack((features, ARA(sample)))
        #features = np.hstack((features, TBP(sample)))
        X_tmp.append(features)

    X = np.array(X_tmp)
    return X

In [7]:
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.2.0-rc2
/device:GPU:0
Num GPUs Available:  1


In [0]:
def Model(num_classes):
  layers = [
     tf.keras.layers.Conv1D(filters=128,kernel_size= 5,strides=2),
      # tf.keras.layers.Conv1D(filters=64,kernel_size= 3,strides=5),
      #   tf.keras.layers.Conv1D(filters=128,kernel_size= 3,strides=2),
      # tf.keras.layers.Conv1D(filters=64,kernel_size= 2,strides=3),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation = tf.nn.relu),
      tf.keras.layers.Dense(64, activation = tf.nn.relu),
      tf.keras.layers.Dense(32, activation = tf.nn.relu),
      tf.keras.layers.Dense(num_classes, activation = 'softmax')
  ]
  model = tf.keras.Sequential(layers)
  return model

In [0]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split

In [0]:
from tensorflow.keras.callbacks import EarlyStopping
def TrainAndReportAccuracy(X,Y,folds,num_classes):
  avg_acc = []
  X_train, X_test, y_train, y_test=train_test_split( X, Y, test_size=0.33, random_state=42)
  X_train = feature_extraction(X_train)
  X_test = feature_extraction(X_test)
  # train_idx = folds[0][0]
  # test_idx = folds[0][1]
  # X_train = X[train_idx]
  # X_test = X[test_idx]
  # Total_Ytrain = Y[train_idx]
  # Total_Ytest = Y[test_idx]
  # Total_Xtrain = feature_extraction(X_train)
  # Total_Xtest = feature_extraction(X_test)
  from sklearn.preprocessing import StandardScaler
  from sklearn.decomposition import PCA
  # for i in range(1, len(folds)):
  #         train_idx = folds[i][0]
  #         test_idx = folds[i][1]

  #         X_train = X[train_idx]
  #         X_test = X[test_idx]
  #         Y_train = Y[train_idx]
  #         Y_test = Y[test_idx]
  #         X_train = feature_extraction(X_train)
  #         X_test = feature_extraction(X_test)
  #         Total_Xtrain=np.append(Total_Xtrain, X_train,axis=0)
  #         Total_Ytrain=np.append(Total_Ytrain, Y_train)
  #         Total_Xtest=np.append(Total_Xtest, X_test,axis=0)
  #         Total_Ytest=np.append(Total_Ytest, Y_test)


  scaler = StandardScaler()
  scaler.fit(X_train)
  X_train = scaler.transform(X_train)
  X_test = scaler.transform(X_test)
  pca = PCA(.95)
  pca.fit(X_train)
  X_train = pca.transform(X_train)
  X_test = pca.transform(X_test)
  print(X_train.shape, y_train.shape)
  print(X_test.shape,y_test.shape)
  size, rows = X_train.shape
  size_test,rows_test=X_test.shape
  X_train=X_train.reshape(X_train.shape[0],rows,1)
  X_test=X_test.reshape(X_test.shape[0],rows_test,1)

  model = Model(num_classes)
  es = EarlyStopping(monitor='sparse_categorical_accuracy', mode='max', verbose=1, patience=50,restore_best_weights=True)
  optimizer = tf.keras.optimizers.Adam(lr=0.001)
    # callbacks = [
    #     tf.keras.callbacks.EarlyStopping(patience=stopping_patience, restore_best_weights=True),
        
    # ]

  model.compile(optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics=[tf.keras.metrics.sparse_categorical_accuracy])
  history = model.fit(X_train, y_train, batch_size=128, epochs=500,callbacks=[es])
  _,acc=model.evaluate(X_test,y_test)
  avg_acc.append(acc)
  print('________________________________________________________________')
  print('Mean Accuracy[{:.4f}] '.format(np.mean(avg_acc)))
  

In [0]:
def Run(data_input_file):
  X,Y,folds,num_classes=DataPreparation(data_input_file)
  TrainAndReportAccuracy(X,Y,folds,num_classes)

In [113]:
Run('data/LOSO/MHEALTH.npz')

Proposed Model data/LOSO/MHEALTH.npz
12
(1711, 22) (1711,)
(844, 22) (844,)
Epoch 1/500
14/14 [==============================] - 0s 2ms/step - loss: 1.8171 - sparse_categorical_accuracy: 0.6961
Epoch 2/500
14/14 [==============================] - 0s 2ms/step - loss: 0.5894 - sparse_categorical_accuracy: 0.9170
Epoch 3/500
14/14 [==============================] - 0s 2ms/step - loss: 0.1524 - sparse_categorical_accuracy: 0.9684
Epoch 4/500
14/14 [==============================] - 0s 2ms/step - loss: 0.0449 - sparse_categorical_accuracy: 0.9936
Epoch 5/500
14/14 [==============================] - 0s 2ms/step - loss: 0.0214 - sparse_categorical_accuracy: 0.9953
Epoch 6/500
14/14 [==============================] - 0s 2ms/step - loss: 0.0116 - sparse_categorical_accuracy: 0.9994
Epoch 7/500
14/14 [==============================] - 0s 2ms/step - loss: 0.0071 - sparse_categorical_accuracy: 1.0000
Epoch 8/500
14/14 [==============================] - 0s 2ms/step - loss: 0.0048 - sparse_categoric

In [114]:
Run('data/LOTO/MHEALTH.npz')

Proposed Model data/LOTO/MHEALTH.npz
12
(1711, 22) (1711,)
(844, 22) (844,)
Epoch 1/500
14/14 [==============================] - 0s 3ms/step - loss: 1.8005 - sparse_categorical_accuracy: 0.6476
Epoch 2/500
14/14 [==============================] - 0s 3ms/step - loss: 0.5376 - sparse_categorical_accuracy: 0.8890
Epoch 3/500
14/14 [==============================] - 0s 3ms/step - loss: 0.1247 - sparse_categorical_accuracy: 0.9755
Epoch 4/500
14/14 [==============================] - 0s 3ms/step - loss: 0.0360 - sparse_categorical_accuracy: 0.9924
Epoch 5/500
14/14 [==============================] - 0s 3ms/step - loss: 0.0158 - sparse_categorical_accuracy: 0.9977
Epoch 6/500
14/14 [==============================] - 0s 3ms/step - loss: 0.0087 - sparse_categorical_accuracy: 0.9994
Epoch 7/500
14/14 [==============================] - 0s 2ms/step - loss: 0.0068 - sparse_categorical_accuracy: 0.9988
Epoch 8/500
14/14 [==============================] - 0s 2ms/step - loss: 0.0054 - sparse_categoric

In [115]:
Run('data/FNOW/MHEALTH.npz')


Proposed Model data/FNOW/MHEALTH.npz
12
(894, 22) (894,)
(441, 22) (441,)
Epoch 1/500
7/7 [==============================] - 0s 3ms/step - loss: 2.2084 - sparse_categorical_accuracy: 0.4128
Epoch 2/500
7/7 [==============================] - 0s 3ms/step - loss: 1.5568 - sparse_categorical_accuracy: 0.7662
Epoch 3/500
7/7 [==============================] - 0s 2ms/step - loss: 0.9563 - sparse_categorical_accuracy: 0.8736
Epoch 4/500
7/7 [==============================] - 0s 2ms/step - loss: 0.4870 - sparse_categorical_accuracy: 0.9262
Epoch 5/500
7/7 [==============================] - 0s 2ms/step - loss: 0.2273 - sparse_categorical_accuracy: 0.9553
Epoch 6/500
7/7 [==============================] - 0s 3ms/step - loss: 0.1233 - sparse_categorical_accuracy: 0.9642
Epoch 7/500
7/7 [==============================] - 0s 3ms/step - loss: 0.0634 - sparse_categorical_accuracy: 0.9877
Epoch 8/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0354 - sparse_categorical_accuracy: 0.992

In [116]:
Run('data/SNOW/MHEALTH.npz')

Proposed Model data/SNOW/MHEALTH.npz
12
(1711, 22) (1711,)
(844, 22) (844,)
Epoch 1/500
14/14 [==============================] - 0s 2ms/step - loss: 1.8520 - sparse_categorical_accuracy: 0.5552
Epoch 2/500
14/14 [==============================] - 0s 2ms/step - loss: 0.6348 - sparse_categorical_accuracy: 0.8545
Epoch 3/500
14/14 [==============================] - 0s 2ms/step - loss: 0.1813 - sparse_categorical_accuracy: 0.9591
Epoch 4/500
14/14 [==============================] - 0s 2ms/step - loss: 0.0471 - sparse_categorical_accuracy: 0.9959
Epoch 5/500
14/14 [==============================] - 0s 2ms/step - loss: 0.0179 - sparse_categorical_accuracy: 0.9982
Epoch 6/500
14/14 [==============================] - 0s 2ms/step - loss: 0.0105 - sparse_categorical_accuracy: 0.9994
Epoch 7/500
14/14 [==============================] - 0s 3ms/step - loss: 0.0068 - sparse_categorical_accuracy: 1.0000
Epoch 8/500
14/14 [==============================] - 0s 2ms/step - loss: 0.0047 - sparse_categoric

In [117]:
Run('data/LOTO/USCHAD.npz')

Proposed Model data/LOTO/USCHAD.npz
12
(6582, 7) (6582,)
(3242, 7) (3242,)
Epoch 1/500
52/52 [==============================] - 0s 2ms/step - loss: 1.4952 - sparse_categorical_accuracy: 0.5146
Epoch 2/500
52/52 [==============================] - 0s 2ms/step - loss: 0.6444 - sparse_categorical_accuracy: 0.7616
Epoch 3/500
52/52 [==============================] - 0s 2ms/step - loss: 0.5101 - sparse_categorical_accuracy: 0.8077
Epoch 4/500
52/52 [==============================] - 0s 2ms/step - loss: 0.4635 - sparse_categorical_accuracy: 0.8194
Epoch 5/500
52/52 [==============================] - 0s 2ms/step - loss: 0.4276 - sparse_categorical_accuracy: 0.8314
Epoch 6/500
52/52 [==============================] - 0s 2ms/step - loss: 0.3975 - sparse_categorical_accuracy: 0.8455
Epoch 7/500
52/52 [==============================] - 0s 2ms/step - loss: 0.3703 - sparse_categorical_accuracy: 0.8555
Epoch 8/500
52/52 [==============================] - 0s 2ms/step - loss: 0.3645 - sparse_categorica

In [118]:
Run('data/LOSO/USCHAD.npz')

Proposed Model data/LOSO/USCHAD.npz
12
(6582, 7) (6582,)
(3242, 7) (3242,)
Epoch 1/500
52/52 [==============================] - 0s 2ms/step - loss: 1.4163 - sparse_categorical_accuracy: 0.5965
Epoch 2/500
52/52 [==============================] - 0s 2ms/step - loss: 0.5996 - sparse_categorical_accuracy: 0.7622
Epoch 3/500
52/52 [==============================] - 0s 3ms/step - loss: 0.4987 - sparse_categorical_accuracy: 0.8081
Epoch 4/500
52/52 [==============================] - 0s 2ms/step - loss: 0.4425 - sparse_categorical_accuracy: 0.8311
Epoch 5/500
52/52 [==============================] - 0s 2ms/step - loss: 0.4050 - sparse_categorical_accuracy: 0.8432
Epoch 6/500
52/52 [==============================] - 0s 2ms/step - loss: 0.3710 - sparse_categorical_accuracy: 0.8541
Epoch 7/500
52/52 [==============================] - 0s 2ms/step - loss: 0.3492 - sparse_categorical_accuracy: 0.8627
Epoch 8/500
52/52 [==============================] - 0s 2ms/step - loss: 0.3343 - sparse_categorica

In [119]:
Run('data/FNOW/USCHAD.npz')

Proposed Model data/FNOW/USCHAD.npz
12
(3375, 7) (3375,)
(1663, 7) (1663,)
Epoch 1/500
27/27 [==============================] - 0s 2ms/step - loss: 1.9750 - sparse_categorical_accuracy: 0.4607
Epoch 2/500
27/27 [==============================] - 0s 2ms/step - loss: 1.0661 - sparse_categorical_accuracy: 0.6720
Epoch 3/500
27/27 [==============================] - 0s 2ms/step - loss: 0.6804 - sparse_categorical_accuracy: 0.7342
Epoch 4/500
27/27 [==============================] - 0s 2ms/step - loss: 0.5757 - sparse_categorical_accuracy: 0.7615
Epoch 5/500
27/27 [==============================] - 0s 2ms/step - loss: 0.5244 - sparse_categorical_accuracy: 0.7953
Epoch 6/500
27/27 [==============================] - 0s 2ms/step - loss: 0.5109 - sparse_categorical_accuracy: 0.8021
Epoch 7/500
27/27 [==============================] - 0s 2ms/step - loss: 0.4713 - sparse_categorical_accuracy: 0.8207
Epoch 8/500
27/27 [==============================] - 0s 2ms/step - loss: 0.4375 - sparse_categorica

In [120]:
Run('data/SNOW/USCHAD.npz')

Proposed Model data/SNOW/USCHAD.npz
12
(6582, 7) (6582,)
(3242, 7) (3242,)
Epoch 1/500
52/52 [==============================] - 0s 2ms/step - loss: 1.4371 - sparse_categorical_accuracy: 0.5548
Epoch 2/500
52/52 [==============================] - 0s 2ms/step - loss: 0.6108 - sparse_categorical_accuracy: 0.7569
Epoch 3/500
52/52 [==============================] - 0s 2ms/step - loss: 0.5238 - sparse_categorical_accuracy: 0.8002
Epoch 4/500
52/52 [==============================] - 0s 2ms/step - loss: 0.4646 - sparse_categorical_accuracy: 0.8186
Epoch 5/500
52/52 [==============================] - 0s 3ms/step - loss: 0.4243 - sparse_categorical_accuracy: 0.8323
Epoch 6/500
52/52 [==============================] - 0s 2ms/step - loss: 0.3942 - sparse_categorical_accuracy: 0.8441
Epoch 7/500
52/52 [==============================] - 0s 2ms/step - loss: 0.3795 - sparse_categorical_accuracy: 0.8467
Epoch 8/500
52/52 [==============================] - 0s 2ms/step - loss: 0.3495 - sparse_categorica

In [121]:
Run('data/LOTO/WHARF.npz')

Proposed Model data/LOTO/WHARF.npz
12
(2593, 5) (2593,)
(1278, 5) (1278,)
Epoch 1/500
21/21 [==============================] - 0s 2ms/step - loss: 2.1514 - sparse_categorical_accuracy: 0.3891
Epoch 2/500
21/21 [==============================] - 0s 2ms/step - loss: 1.5130 - sparse_categorical_accuracy: 0.5040
Epoch 3/500
21/21 [==============================] - 0s 2ms/step - loss: 1.2443 - sparse_categorical_accuracy: 0.5600
Epoch 4/500
21/21 [==============================] - 0s 2ms/step - loss: 1.1301 - sparse_categorical_accuracy: 0.5804
Epoch 5/500
21/21 [==============================] - 0s 2ms/step - loss: 1.0710 - sparse_categorical_accuracy: 0.6089
Epoch 6/500
21/21 [==============================] - 0s 2ms/step - loss: 1.0122 - sparse_categorical_accuracy: 0.6244
Epoch 7/500
21/21 [==============================] - 0s 2ms/step - loss: 0.9762 - sparse_categorical_accuracy: 0.6386
Epoch 8/500
21/21 [==============================] - 0s 2ms/step - loss: 0.9445 - sparse_categorical

In [122]:
Run('data/LOSO/WHARF.npz')

Proposed Model data/LOSO/WHARF.npz
12
(2593, 5) (2593,)
(1278, 5) (1278,)
Epoch 1/500
21/21 [==============================] - 0s 2ms/step - loss: 2.0764 - sparse_categorical_accuracy: 0.4288
Epoch 2/500
21/21 [==============================] - 0s 2ms/step - loss: 1.5287 - sparse_categorical_accuracy: 0.4948
Epoch 3/500
21/21 [==============================] - 0s 2ms/step - loss: 1.3446 - sparse_categorical_accuracy: 0.5299
Epoch 4/500
21/21 [==============================] - 0s 2ms/step - loss: 1.2225 - sparse_categorical_accuracy: 0.5673
Epoch 5/500
21/21 [==============================] - 0s 2ms/step - loss: 1.1358 - sparse_categorical_accuracy: 0.5800
Epoch 6/500
21/21 [==============================] - 0s 2ms/step - loss: 1.0571 - sparse_categorical_accuracy: 0.6155
Epoch 7/500
21/21 [==============================] - 0s 2ms/step - loss: 0.9948 - sparse_categorical_accuracy: 0.6352
Epoch 8/500
21/21 [==============================] - 0s 2ms/step - loss: 0.9611 - sparse_categorical

In [123]:
Run('data/SNOW/WHARF.npz')

Proposed Model data/SNOW/WHARF.npz
12
(2599, 5) (2599,)
(1281, 5) (1281,)
Epoch 1/500
21/21 [==============================] - 0s 2ms/step - loss: 2.0969 - sparse_categorical_accuracy: 0.4205
Epoch 2/500
21/21 [==============================] - 0s 2ms/step - loss: 1.5717 - sparse_categorical_accuracy: 0.4940
Epoch 3/500
21/21 [==============================] - 0s 2ms/step - loss: 1.3559 - sparse_categorical_accuracy: 0.5348
Epoch 4/500
21/21 [==============================] - 0s 2ms/step - loss: 1.2368 - sparse_categorical_accuracy: 0.5756
Epoch 5/500
21/21 [==============================] - 0s 2ms/step - loss: 1.1441 - sparse_categorical_accuracy: 0.5860
Epoch 6/500
21/21 [==============================] - 0s 2ms/step - loss: 1.0728 - sparse_categorical_accuracy: 0.5975
Epoch 7/500
21/21 [==============================] - 0s 2ms/step - loss: 1.0083 - sparse_categorical_accuracy: 0.6179
Epoch 8/500
21/21 [==============================] - 0s 2ms/step - loss: 0.9570 - sparse_categorical

In [124]:
Run('data/FNOW/WHARF.npz')

Proposed Model data/FNOW/WHARF.npz
12
(1437, 5) (1437,)
(709, 5) (709,)
Epoch 1/500
12/12 [==============================] - 0s 2ms/step - loss: 2.2710 - sparse_categorical_accuracy: 0.3702
Epoch 2/500
12/12 [==============================] - 0s 2ms/step - loss: 1.8840 - sparse_categorical_accuracy: 0.4614
Epoch 3/500
12/12 [==============================] - 0s 2ms/step - loss: 1.5839 - sparse_categorical_accuracy: 0.4718
Epoch 4/500
12/12 [==============================] - 0s 3ms/step - loss: 1.4523 - sparse_categorical_accuracy: 0.5379
Epoch 5/500
12/12 [==============================] - 0s 2ms/step - loss: 1.3140 - sparse_categorical_accuracy: 0.5351
Epoch 6/500
12/12 [==============================] - 0s 2ms/step - loss: 1.2256 - sparse_categorical_accuracy: 0.5546
Epoch 7/500
12/12 [==============================] - 0s 2ms/step - loss: 1.1926 - sparse_categorical_accuracy: 0.5685
Epoch 8/500
12/12 [==============================] - 0s 2ms/step - loss: 1.1335 - sparse_categorical_a

In [125]:
Run('data/LOTO/WISDM.npz')

Proposed Model data/LOTO/WISDM.npz
6
(13966, 5) (13966,)
(6880, 5) (6880,)
Epoch 1/500
110/110 [==============================] - 0s 2ms/step - loss: 0.7592 - sparse_categorical_accuracy: 0.7562
Epoch 2/500
110/110 [==============================] - 0s 2ms/step - loss: 0.4386 - sparse_categorical_accuracy: 0.8346
Epoch 3/500
110/110 [==============================] - 0s 2ms/step - loss: 0.3816 - sparse_categorical_accuracy: 0.8569
Epoch 4/500
110/110 [==============================] - 0s 2ms/step - loss: 0.3492 - sparse_categorical_accuracy: 0.8689
Epoch 5/500
110/110 [==============================] - 0s 2ms/step - loss: 0.3276 - sparse_categorical_accuracy: 0.8782
Epoch 6/500
110/110 [==============================] - 0s 2ms/step - loss: 0.3100 - sparse_categorical_accuracy: 0.8889
Epoch 7/500
110/110 [==============================] - 0s 3ms/step - loss: 0.2997 - sparse_categorical_accuracy: 0.8902
Epoch 8/500
110/110 [==============================] - 0s 3ms/step - loss: 0.2830 - s

In [126]:
Run('data/LOSO/WISDM.npz')

Proposed Model data/LOSO/WISDM.npz
6
(13966, 5) (13966,)
(6880, 5) (6880,)
Epoch 1/500
110/110 [==============================] - 0s 2ms/step - loss: 0.7760 - sparse_categorical_accuracy: 0.7601
Epoch 2/500
110/110 [==============================] - 0s 2ms/step - loss: 0.4510 - sparse_categorical_accuracy: 0.8347
Epoch 3/500
110/110 [==============================] - 0s 2ms/step - loss: 0.3952 - sparse_categorical_accuracy: 0.8538
Epoch 4/500
110/110 [==============================] - 0s 2ms/step - loss: 0.3618 - sparse_categorical_accuracy: 0.8687
Epoch 5/500
110/110 [==============================] - 0s 2ms/step - loss: 0.3366 - sparse_categorical_accuracy: 0.8763
Epoch 6/500
110/110 [==============================] - 0s 2ms/step - loss: 0.3146 - sparse_categorical_accuracy: 0.8832
Epoch 7/500
110/110 [==============================] - 0s 2ms/step - loss: 0.3034 - sparse_categorical_accuracy: 0.8872
Epoch 8/500
110/110 [==============================] - 0s 2ms/step - loss: 0.2922 - s

In [127]:
Run('data/SNOW/WISDM.npz')

Proposed Model data/SNOW/WISDM.npz
6
(13966, 5) (13966,)
(6880, 5) (6880,)
Epoch 1/500
110/110 [==============================] - 0s 2ms/step - loss: 0.7427 - sparse_categorical_accuracy: 0.7687
Epoch 2/500
110/110 [==============================] - 0s 2ms/step - loss: 0.4426 - sparse_categorical_accuracy: 0.8330
Epoch 3/500
110/110 [==============================] - 0s 2ms/step - loss: 0.3967 - sparse_categorical_accuracy: 0.8521
Epoch 4/500
110/110 [==============================] - 0s 2ms/step - loss: 0.3624 - sparse_categorical_accuracy: 0.8668
Epoch 5/500
110/110 [==============================] - 0s 2ms/step - loss: 0.3386 - sparse_categorical_accuracy: 0.8768
Epoch 6/500
110/110 [==============================] - 0s 2ms/step - loss: 0.3228 - sparse_categorical_accuracy: 0.8802
Epoch 7/500
110/110 [==============================] - 0s 2ms/step - loss: 0.3064 - sparse_categorical_accuracy: 0.8857
Epoch 8/500
110/110 [==============================] - 0s 2ms/step - loss: 0.2828 - s

In [128]:
Run('data/FNOW/WISDM.npz')

Proposed Model data/FNOW/WISDM.npz
6
(7045, 5) (7045,)
(3471, 5) (3471,)
Epoch 1/500
56/56 [==============================] - 0s 2ms/step - loss: 0.9957 - sparse_categorical_accuracy: 0.7155
Epoch 2/500
56/56 [==============================] - 0s 2ms/step - loss: 0.5737 - sparse_categorical_accuracy: 0.7979
Epoch 3/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4747 - sparse_categorical_accuracy: 0.8202
Epoch 4/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4465 - sparse_categorical_accuracy: 0.8321
Epoch 5/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4173 - sparse_categorical_accuracy: 0.8440
Epoch 6/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4064 - sparse_categorical_accuracy: 0.8508
Epoch 7/500
56/56 [==============================] - 0s 2ms/step - loss: 0.3969 - sparse_categorical_accuracy: 0.8491
Epoch 8/500
56/56 [==============================] - 0s 3ms/step - loss: 0.3681 - sparse_categorical_

In [129]:
Run('data/LOTO/UTD-MHAD2_1s.npz')

Proposed Model data/LOTO/UTD-MHAD2_1s.npz
6
(761, 8) (761,)
(376, 8) (376,)
Epoch 1/500
6/6 [==============================] - 0s 3ms/step - loss: 1.6392 - sparse_categorical_accuracy: 0.3206
Epoch 2/500
6/6 [==============================] - 0s 2ms/step - loss: 1.3820 - sparse_categorical_accuracy: 0.5296
Epoch 3/500
6/6 [==============================] - 0s 2ms/step - loss: 1.1758 - sparse_categorical_accuracy: 0.5926
Epoch 4/500
6/6 [==============================] - 0s 2ms/step - loss: 0.9971 - sparse_categorical_accuracy: 0.6597
Epoch 5/500
6/6 [==============================] - 0s 2ms/step - loss: 0.8734 - sparse_categorical_accuracy: 0.6794
Epoch 6/500
6/6 [==============================] - 0s 2ms/step - loss: 0.7964 - sparse_categorical_accuracy: 0.6846
Epoch 7/500
6/6 [==============================] - 0s 2ms/step - loss: 0.7422 - sparse_categorical_accuracy: 0.6912
Epoch 8/500
6/6 [==============================] - 0s 2ms/step - loss: 0.7012 - sparse_categorical_accuracy: 0.7

In [130]:
Run('data/LOSO/UTD-MHAD2_1s.npz')

Proposed Model data/LOSO/UTD-MHAD2_1s.npz
6
(761, 8) (761,)
(376, 8) (376,)
Epoch 1/500
6/6 [==============================] - 0s 3ms/step - loss: 1.6893 - sparse_categorical_accuracy: 0.3758
Epoch 2/500
6/6 [==============================] - 0s 3ms/step - loss: 1.4530 - sparse_categorical_accuracy: 0.6045
Epoch 3/500
6/6 [==============================] - 0s 3ms/step - loss: 1.2416 - sparse_categorical_accuracy: 0.6281
Epoch 4/500
6/6 [==============================] - 0s 2ms/step - loss: 1.0849 - sparse_categorical_accuracy: 0.6281
Epoch 5/500
6/6 [==============================] - 0s 2ms/step - loss: 0.9484 - sparse_categorical_accuracy: 0.6649
Epoch 6/500
6/6 [==============================] - 0s 3ms/step - loss: 0.8389 - sparse_categorical_accuracy: 0.6807
Epoch 7/500
6/6 [==============================] - 0s 2ms/step - loss: 0.7405 - sparse_categorical_accuracy: 0.7254
Epoch 8/500
6/6 [==============================] - 0s 2ms/step - loss: 0.6818 - sparse_categorical_accuracy: 0.7

In [131]:
Run('data/FNOW/UTD-MHAD2_1s.npz')

Proposed Model data/FNOW/UTD-MHAD2_1s.npz
6
(412, 7) (412,)
(204, 7) (204,)
Epoch 1/500
4/4 [==============================] - 0s 3ms/step - loss: 1.7249 - sparse_categorical_accuracy: 0.3350
Epoch 2/500
4/4 [==============================] - 0s 3ms/step - loss: 1.5136 - sparse_categorical_accuracy: 0.5024
Epoch 3/500
4/4 [==============================] - 0s 3ms/step - loss: 1.3551 - sparse_categorical_accuracy: 0.5316
Epoch 4/500
4/4 [==============================] - 0s 3ms/step - loss: 1.2199 - sparse_categorical_accuracy: 0.5437
Epoch 5/500
4/4 [==============================] - 0s 2ms/step - loss: 1.0930 - sparse_categorical_accuracy: 0.5388
Epoch 6/500
4/4 [==============================] - 0s 3ms/step - loss: 0.9259 - sparse_categorical_accuracy: 0.5874
Epoch 7/500
4/4 [==============================] - 0s 3ms/step - loss: 0.9049 - sparse_categorical_accuracy: 0.6553
Epoch 8/500
4/4 [==============================] - 0s 3ms/step - loss: 0.8787 - sparse_categorical_accuracy: 0.6

In [132]:
Run('data/SNOW/UTD-MHAD2_1s.npz')

Proposed Model data/SNOW/UTD-MHAD2_1s.npz
6
(761, 8) (761,)
(376, 8) (376,)
Epoch 1/500
6/6 [==============================] - 0s 3ms/step - loss: 1.6857 - sparse_categorical_accuracy: 0.4244
Epoch 2/500
6/6 [==============================] - 0s 2ms/step - loss: 1.4303 - sparse_categorical_accuracy: 0.5861
Epoch 3/500
6/6 [==============================] - 0s 2ms/step - loss: 1.1981 - sparse_categorical_accuracy: 0.6163
Epoch 4/500
6/6 [==============================] - 0s 3ms/step - loss: 1.0265 - sparse_categorical_accuracy: 0.6491
Epoch 5/500
6/6 [==============================] - 0s 2ms/step - loss: 0.9072 - sparse_categorical_accuracy: 0.6675
Epoch 6/500
6/6 [==============================] - 0s 3ms/step - loss: 0.8190 - sparse_categorical_accuracy: 0.6886
Epoch 7/500
6/6 [==============================] - 0s 3ms/step - loss: 0.7640 - sparse_categorical_accuracy: 0.6978
Epoch 8/500
6/6 [==============================] - 0s 3ms/step - loss: 0.7217 - sparse_categorical_accuracy: 0.7

In [133]:
Run('data/LOTO/UTD-MHAD1_1s.npz')

Proposed Model data/LOTO/UTD-MHAD1_1s.npz
21
(2526, 10) (2526,)
(1245, 10) (1245,)
Epoch 1/500
20/20 [==============================] - 0s 2ms/step - loss: 2.9375 - sparse_categorical_accuracy: 0.1148
Epoch 2/500
20/20 [==============================] - 0s 2ms/step - loss: 2.5851 - sparse_categorical_accuracy: 0.2379
Epoch 3/500
20/20 [==============================] - 0s 3ms/step - loss: 2.1832 - sparse_categorical_accuracy: 0.3452
Epoch 4/500
20/20 [==============================] - 0s 2ms/step - loss: 1.9144 - sparse_categorical_accuracy: 0.4153
Epoch 5/500
20/20 [==============================] - 0s 2ms/step - loss: 1.7294 - sparse_categorical_accuracy: 0.4580
Epoch 6/500
20/20 [==============================] - 0s 3ms/step - loss: 1.6233 - sparse_categorical_accuracy: 0.4917
Epoch 7/500
20/20 [==============================] - 0s 2ms/step - loss: 1.5266 - sparse_categorical_accuracy: 0.5273
Epoch 8/500
20/20 [==============================] - 0s 2ms/step - loss: 1.4491 - sparse_ca

In [134]:
Run('data/LOSO/UTD-MHAD1_1s.npz')

Proposed Model data/LOSO/UTD-MHAD1_1s.npz
21
(2526, 10) (2526,)
(1245, 10) (1245,)
Epoch 1/500
20/20 [==============================] - 0s 3ms/step - loss: 2.9366 - sparse_categorical_accuracy: 0.1386
Epoch 2/500
20/20 [==============================] - 0s 3ms/step - loss: 2.6234 - sparse_categorical_accuracy: 0.2169
Epoch 3/500
20/20 [==============================] - 0s 3ms/step - loss: 2.2476 - sparse_categorical_accuracy: 0.3195
Epoch 4/500
20/20 [==============================] - 0s 3ms/step - loss: 1.9452 - sparse_categorical_accuracy: 0.4014
Epoch 5/500
20/20 [==============================] - 0s 3ms/step - loss: 1.7442 - sparse_categorical_accuracy: 0.4454
Epoch 6/500
20/20 [==============================] - 0s 3ms/step - loss: 1.6148 - sparse_categorical_accuracy: 0.4834
Epoch 7/500
20/20 [==============================] - 0s 2ms/step - loss: 1.5234 - sparse_categorical_accuracy: 0.5198
Epoch 8/500
20/20 [==============================] - 0s 2ms/step - loss: 1.4348 - sparse_ca

In [135]:
Run('data/SNOW/UTD-MHAD1_1s.npz')

Proposed Model data/SNOW/UTD-MHAD1_1s.npz
21
(2526, 10) (2526,)
(1245, 10) (1245,)
Epoch 1/500
20/20 [==============================] - 0s 3ms/step - loss: 2.8967 - sparse_categorical_accuracy: 0.1540
Epoch 2/500
20/20 [==============================] - 0s 2ms/step - loss: 2.5440 - sparse_categorical_accuracy: 0.2415
Epoch 3/500
20/20 [==============================] - 0s 2ms/step - loss: 2.1501 - sparse_categorical_accuracy: 0.3603
Epoch 4/500
20/20 [==============================] - 0s 2ms/step - loss: 1.8996 - sparse_categorical_accuracy: 0.3955
Epoch 5/500
20/20 [==============================] - 0s 2ms/step - loss: 1.7414 - sparse_categorical_accuracy: 0.4501
Epoch 6/500
20/20 [==============================] - 0s 2ms/step - loss: 1.6214 - sparse_categorical_accuracy: 0.4834
Epoch 7/500
20/20 [==============================] - 0s 2ms/step - loss: 1.5248 - sparse_categorical_accuracy: 0.5146
Epoch 8/500
20/20 [==============================] - 0s 2ms/step - loss: 1.4481 - sparse_ca

In [136]:
Run('data/FNOW/UTD-MHAD1_1s.npz')

Proposed Model data/FNOW/UTD-MHAD1_1s.npz
21
(1372, 10) (1372,)
(676, 10) (676,)
Epoch 1/500
11/11 [==============================] - 0s 2ms/step - loss: 2.9699 - sparse_categorical_accuracy: 0.1261
Epoch 2/500
11/11 [==============================] - 0s 2ms/step - loss: 2.7672 - sparse_categorical_accuracy: 0.2420
Epoch 3/500
11/11 [==============================] - 0s 2ms/step - loss: 2.4924 - sparse_categorical_accuracy: 0.2806
Epoch 4/500
11/11 [==============================] - 0s 2ms/step - loss: 2.2324 - sparse_categorical_accuracy: 0.3462
Epoch 5/500
11/11 [==============================] - 0s 2ms/step - loss: 1.9995 - sparse_categorical_accuracy: 0.3994
Epoch 6/500
11/11 [==============================] - 0s 2ms/step - loss: 1.8178 - sparse_categorical_accuracy: 0.4431
Epoch 7/500
11/11 [==============================] - 0s 2ms/step - loss: 1.6807 - sparse_categorical_accuracy: 0.4789
Epoch 8/500
11/11 [==============================] - 0s 2ms/step - loss: 1.5684 - sparse_cate